In [1]:
!pip install -q rouge_score bert_score

In [2]:
from transformers import MBartForConditionalGeneration, AutoModelForSeq2SeqLM
from transformers import MT5ForConditionalGeneration, MT5Tokenizer
from transformers import AlbertTokenizer, AutoTokenizer, pipeline
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import load_metric
from tqdm import tqdm
from bert_score import score

2024-04-25 05:40:22.566569: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-25 05:40:22.566668: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-25 05:40:22.692307: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
# tokenizer = AutoTokenizer.from_pretrained("ai4bharat/IndicBART", do_lower_case=False, use_fast=False, keep_accents=True)
# tokenizer_SS = AutoTokenizer.from_pretrained("ai4bharat/IndicBARTSS", do_lower_case=False, use_fast=False, keep_accents=True)

# model = AutoModelForSeq2SeqLM.from_pretrained("ai4bharat/IndicBART").to('cuda')
# model_SS = AutoModelForSeq2SeqLM.from_pretrained("ai4bharat/IndicBARTSS").to('cuda')

tokenizer = MT5Tokenizer.from_pretrained("google/mt5-small")
model = MT5ForConditionalGeneration.from_pretrained("google/mt5-small").to('cuda')

tokenizer_config.json:   0%|          | 0.00/82.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/553 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


pytorch_model.bin:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [4]:
train_data = pd.read_csv("/kaggle/input/hindidataset/HindiNews/HindiNews_train_v2/hindi_train.csv")
train_data_subset = train_data.head(10000)
train_set, test_set = train_test_split(train_data, test_size=0.2, shuffle=False)
sample = train_set.loc[0]

In [5]:
maxx = 0
cnt = 0
lengths = [len(summ) for summ in train_data_subset['Summary']]
print(max(lengths))
print(sum(lengths)/len(lengths))

412
224.6661


In [6]:
def generate_summary(model, text):
    inputs = tokenizer(text, return_tensors="pt", max_length=1024, truncation=True).to('cuda')
    summary_ids = model.generate(inputs["input_ids"], num_beams=4, max_length=50, early_stopping=False)
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

In [7]:
generated_summaries = []
# generated_summaries_SS = []
for _, example in tqdm(train_data_subset.iterrows(), total=len(train_data_subset)):
#     a = (len(example["Summary"]))
    summary = generate_summary(model, example["Article"])
    generated_summaries.append(summary)
#     summary_SS = generate_summary(model_SS, example["Article"])
#     generated_summaries_SS.append(summary_SS)

100%|██████████| 10000/10000 [42:43<00:00,  3.90it/s]


In [8]:
metric = load_metric("rouge")
references = [example["Summary"] for _, example in train_data_subset.iterrows()]
rouge_scores = metric.compute(predictions=generated_summaries, references=references)
# rouge_scores_SS = metric.compute(predictions=generated_summaries_SS, references=references)

/tmp/ipykernel_24/3373130066.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("rouge")
/opt/conda/lib/python3.10/site-packages/datasets/load.py:756: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [9]:
P, R, F1 = score(generated_summaries, references, lang='hi', verbose=True)
# P_SS, R_SS, F1_SS = score(generated_summaries_SS, references, lang='hi', verbose=True)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

calculating scores...
computing bert embedding.


  0%|          | 0/268 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/157 [00:00<?, ?it/s]

done in 55.64 seconds, 179.73 sentences/sec


In [10]:
with open("/kaggle/working/rouge_scores.txt", "w") as f:
    f.write(str(rouge_scores))

# with open("/kaggle/working/rouge_scores_SS.txt", "w") as f:
#     f.write(str(rouge_scores_SS))

with open("/kaggle/working/bert_scores.txt", "w") as f:
    f.write(f"P: {P.mean()}\nR: {R.mean()}\nF1: {F1.mean()}")

# with open("/kaggle/working/bert_scores_SS.txt", "w") as f:
#     f.write(f"P: {P_SS.mean()}\nR: {R_SS.mean()}\nF1: {F1_SS.mean()}")